# Lorenzo helper functions for data analysis

In [15]:
# Reference pixel to look for correlations
ref_row = 100
ref_col = 200

# Filter bad pixels
def filter_bad_pixels(data):

    noise = np.empty([np.shape(data)[0],np.shape(data)[1]])
    for row, col in np.ndindex(np.shape(data)[0],np.shape(data)[1]):
        # print(row,col)
        noise[row,col] = np.std(data[row,col,:])

    noise_median = np.median(noise)

    for row, col in np.ndindex(np.shape(data)[0],np.shape(data)[1]):
        if noise[row,col] > 3*noise_median:
            print('Found bad pixel {}:{}'.format(row,col))
            data[row,col,:] = np.median(data)

    return data, noise_median

# Reject outliers
def rejOutliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

# Correlation matrix
def calculate_correlation(data):

    corr_mat   = np.empty([np.shape(data)[0],np.shape(data)[1]])
    
    for row, col in np.ndindex(np.shape(data)[0],np.shape(data)[1]):
        # Correlation coefficients
        corr_mat[row,col] = np.corrcoef(data[row,col,:], data[ref_row, ref_col,:])[0,1]

    corr_mat[ref_row,ref_col] = np.average(corr_mat)
    return corr_mat


# Configure charge injection
def chargeInjection(firstCol=120,LastCol=180,PulserValue=100,AsicNum=3):
    APP.prepareChargeInjection(3, firstCol, LastCol, PulserValue)

# Readout frame(s)
def readoutFrames(numberOfTriggers=1,chargeInj=False):
    DEBUG3.cleanData()

    for TrigNum in range(numberOfTriggers):
        if (chargeInj != False):
            chargeInjection(PulserValue=chargeInj)
        root.Trigger()
        print("{}".format(TrigNum+1), end='\r')
        time.sleep(0.01)

    # Although the triggers above may have finished, descrambling takes time
    while ( numberOfTriggers != DEBUG3.getData().shape[2]) :
        time.sleep(0.1)
        print("Descrambled {}".format(DEBUG3.getData().shape[2]), end='\r')
    print("Data Descrambled")
    print(DEBUG3.getData().shape)

    # Data format = [row,col,frame_number]
    data = DEBUG3.getData()
    DEBUG3.cleanData()
    return data
    
# Get Average Dark value over rows/cols
def getMedianDark(data):
    return np.median(np.median(data,axis=(0,1)))

# Calculate gain
def ChInjRamp(numberOfTriggers=3,Min=100,Max=150,numSteps=10):

    ChInjValues = np.linspace(start=Min, stop=Max, num=numSteps)
    PixelOutValues = np.empty(np.shape(ChInjValues))
      
    for i, PulserValue in enumerate(ChInjValues):
        # Get frames but look only at ref pixel
        data_temp = readoutFrames(numberOfTriggers,chargeInj=PulserValue)[ref_row,ref_col,:]
        # Get median across frames and add to results array
        data_temp = np.median(data_temp)
        PixelOutValues[i] = data_temp
    
    return ChInjValues, PixelOutValues
  
    
    